# Debt simplification

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=examples/debt-simplification.ipynb">directly from your browser</a>.
</div>

A [mathematical programming](https://en.wikipedia.org/wiki/Mathematical_optimization) approach for settling debts within a group, similar to [Splitwise's debt simplification](https://blog.splitwise.com/2012/09/14/debts-made-simple/).

In [1]:
%pip install opvious

## Problem formulation

We first define our model using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [2]:
import opvious.modeling as om

class GroupExpenses(om.Model):
    """A mixed-integer model for settling debts within a group
    
    The solution will represent the optimal transfers between group members in order to achieve fairness: each
    member will end up having paid a total amount proportional to their involvement in the group's transactions.
    """
    
    members = om.Dimension()  # Participants
    transactions = om.Dimension()  # Expenses
    is_participating = om.Parameter.indicator(transactions, members)  # 1 if a member is involved in a transaction
    payment = om.Parameter.non_negative(transactions, members)  # Amount paid by each member per transaction

    # Amount to be transferred by one member to another to achieve fairness
    transfer = om.Variable.non_negative(members, members, qualifiers=['sender', 'recipient'])
    # Indicator variable representing a transfer from one member to another (1 if transfer > 0, 0 otherwise)
    is_transferring = om.fragments.ActivationVariable(transfer, upper_bound=payment.total())

    def fair_payment(self, t, m):
        """Fair payment in a transaction for a given member"""
        share = self.is_participating(t, m) / om.total(self.is_participating(t, o) for o in self.members)
        return share * om.total(self.payment(t, m) for m in self.members)
    
    @om.constraint
    def zero_sum_transfers(self):
        """After netting transfers, each member should have paid the sum of their fair payments"""
        for m in self.members:
            received = om.total(self.transfer(s, m) for s in self.members)
            sent = om.total(self.transfer(m, r) for r in self.members)
            owed = om.total(self.payment(t, m) - self.fair_payment(t, m) for t in self.transactions)
            yield received - sent == owed
            
    @om.objective
    def minimize_total_transferred(self):
        """First objective: minimize the total amount of money transferred between members"""
        return om.total(self.transfer(s, r) for s, r in self.members * self.members)
    
    @om.fragments.magnitude_variable(members, projection=0, lower_bound=False)
    def max_transfers_sent(self, m):
        """Number of transfers sent by any member"""
        return om.total(self.is_transferring(m, r) for r in self.members)
    
    @om.objective
    def minimize_max_transfers_sent(self):
        """Second objective: minimize the maximum number of transfers sent by any member"""
        return self.max_transfers_sent()


model = GroupExpenses()
model.definition_counts().T # Summary of the model's components

title,GroupExpenses
category,
CONSTRAINT,3
DIMENSION,2
OBJECTIVE,2
PARAMETER,2
VARIABLE,3


For the mathematically inclined, models can generate their LaTeX specification.

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">GroupExpenses</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{members}&: M \\
  \S^d_\mathrm{transactions}&: T \\
  \S^p_\mathrm{isParticipating}&: p^\mathrm{is} \in \{0, 1\}^{T \times M} \\
  \S^p_\mathrm{payment}&: p \in \mathbb{R}_+^{T \times M} \\
  \S^v_\mathrm{transfer[sender,recipient]}&: \tau \in \mathbb{R}_+^{M \times M} \\
  \S^v_\mathrm{isTransferring}&: \tau^\mathrm{is} \in \{0, 1\}^{M \times M} \\
  \S^c_\mathrm{isTransferringActivates}&: \forall m, m' \in M, \sum_{t \in T, m'' \in M} p_{t,m''} \tau^\mathrm{is}_{m,m'} \geq \tau_{m,m'} \\
  \S^c_\mathrm{zeroSumTransfers}&: \forall m \in M, \sum_{m' \in M} \tau_{m',m} - \sum_{m' \in M} \tau_{m,m'} = \sum_{t \in T} \left(p_{t,m} - \frac{p^\mathrm{is}_{t,m}}{\sum_{m' \in M} p^\mathrm{is}_{t,m'}} \sum_{m' \in M} p_{t,m'}\right) \\
  \S^o_\mathrm{minimizeTotalTransferred}&: \min \sum_{m, m' \in M} \tau_{m,m'} \\
  \S^v_\mathrm{maxTransfersSent}&: \sigma^\mathrm{maxTransfers} \in \mathbb{R}_+ \\
  \S^c_\mathrm{maxTransfersSentUpperBounds}&: \forall m \in M, \sigma^\mathrm{maxTransfers} \geq \sum_{m' \in M} \tau^\mathrm{is}_{m,m'} \\
  \S^o_\mathrm{minimizeMaxTransfersSent}&: \min \sigma^\mathrm{maxTransfers} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

We wrap the formulation defined above into a simple function which returns the optimal transfers given input data.

A few things to note:

+ Solves run remotely--no local solver installation required--and can be configured via `opvious` [client](https://opvious.readthedocs.io/en/stable/overview.html#creating-a-client) instances.
+ We leverage `pandas` utilities directly thanks to the SDK's native support for dataframes.
+ We specify a custom [multi-objective strategy](https://opvious.readthedocs.io/en/stable/strategies.html) to efficiently pick a robust optimal solution.

In [4]:
import opvious

async def compute_optimal_transfers(payments, tolerance=0.1):
    """Computes optimal transfers to settle expenses fairly within a group
    
    Args:
        payments: Dataframe of payments indexed by transaction where each column is a group member.
            Each member with a non-zero payment will be considered a participant in the transaction.
        tolerance: Relative slack bound on the total amount of money transferred during settlement
            used to minimize the number of outbound transfers for any one member. For example, the
            default value of 0.1 will allow transfering up to 10% more overall.
    """
    se = payments.stack()['payment']  # Payments keyed by (transaction, member)
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={
            'payment': se,
            'isParticipating': (se > 0).astype(int),
        },
        strategy=opvious.SolveStrategy(  # Multi-objective strategy
            epsilon_constraints=[
                # Within tolerance of the smallest total transfer amount
                opvious.EpsilonConstraint('minimizeTotalTransferred', relative_tolerance=tolerance),
                # Using the smallest possible number of transfers per member
                opvious.EpsilonConstraint('minimizeMaxTransfersSent'),
            ],
            target='minimizeTotalTransferred',  # Final target: minimize total transfer amount
        ),
    )
    client = opvious.Client.from_environment(default_endpoint=opvious.DEMO_ENDPOINT)
    solution = await client.solve(problem)
    return solution.outputs.variable('transfer').unstack(level=1).fillna(0).round(2)

## Testing

We test our implementation on some representative data.

In [5]:
import logging
import numpy as np
import pandas as pd

logging.basicConfig(level=logging.INFO)

_names = ["emma", "noah", "ava", "liam", "isabella", "sophia", "mason", "mia", "lucas"]

def generate_random_payments(count=25, seed=2):
    """Generates a random dataframe of non-negative payments"""
    rng = np.random.default_rng(seed)
    tuples = []
    for i in range(count):
        tid = f't{i+1:02}'
        for name in _names:
            if rng.integers(2):
                continue
            tuples.append({
                'transaction': tid, 
                'name': name, 
                'payment': round(100 * rng.random(), 2),
            })
    df = pd.DataFrame(tuples).set_index(['transaction', 'name'])
    return df.unstack().fillna(0)

payments_df = generate_random_payments()
payments_df.head()

payment                                                         
name            ava   emma isabella   liam  lucas  mason   mia   noah sophia
transaction                                                                 
t01            9.19   0.00    72.86   0.00   0.00   0.00  5.51  29.85   0.00
t02            0.00  65.74    66.93   0.00   0.00  63.32  0.00  15.01   0.00
t03           18.73   0.00     0.00  34.60   0.00   0.00  0.00   0.00   0.00
t04           92.42   0.00    10.72  69.38   0.00  88.44  0.00   0.00  20.19
t05           51.66  64.44    43.82  59.34  49.81  61.37  0.00   0.00   0.00

Using the default tolerance of 10%, we get the following optimals transfers:

In [6]:
await compute_optimal_transfers(payments_df)

INFO:opvious.client.handlers:Validated inputs. [parameters=450]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=99]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=358.48607142857134]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=8, gap=100.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=45, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=159, gap=75.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=267, gap=50.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=2]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=21, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=191, gap=3.37%]
INFO:opvious.cl

value                                      
recipient   emma isabella   liam  lucas   mason   noah
sender                                                
ava         0.00    36.84  12.35   0.00    0.00   0.00
liam        2.34     0.00   0.00   0.00    1.52   0.00
mia         0.00     0.00   0.00   0.00  162.25  16.36
sophia     48.66     0.00   0.00  82.03    0.00   0.00

In the solution above, the total amount of money transferred comes up to ~$362 and each person sends at most 2 transfers (Ava, Liam, Mia, and Sophia all send 2).

Let's see what happens if we reduce the tolerance to 0, forcing the solution to focus on minimizing total transfer amount.

In [7]:
await compute_optimal_transfers(payments_df, tolerance=0)

INFO:opvious.client.handlers:Validated inputs. [parameters=450]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=99]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=358.48607142857134]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=8, gap=100.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=44, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=194, gap=83.33%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=3]
INFO:opvious.client.handlers:Solve in progress... [iterations=349, gap=0.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=21, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=60, gap=0.0%]
INFO:opvious.clie

value                                     
recipient  emma isabella  liam  lucas   mason   noah
sender                                              
ava        18.7    30.49  0.00   0.00    0.00   0.00
mia         0.0     6.35  8.48   0.00  163.77   0.00
sophia     32.3     0.00  0.00  82.03    0.00  16.36

As expected the total transfer amount is lower (slightly, ~$358), however our other objective has increased: Mia and Sophia send three transfers each.

Let's try increasing the tolerance instead to find a solution where each person sends at most a single transfer.

In [8]:
await compute_optimal_transfers(payments_df, tolerance=0.25)

INFO:opvious.client.handlers:Validated inputs. [parameters=450]
INFO:opvious.client.handlers:Solving problem... [columns=163, rows=99]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=358.48607142857134]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=8, gap=100.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=42, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=135, gap=75.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=307, gap=50.0%]
INFO:opvious.client.handlers:Added epsilon constraint. [objective_value=1]
INFO:opvious.client.handlers:Solve in progress... [iterations=779, gap=0.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=n/a]
INFO:opvious.client.handlers:Solve in progress... [iterations=22, gap=n/a]
INFO:opvious.cli

value                                       
recipient   emma isabella   liam   lucas   mason   noah
sender                                                 
ava        49.19     0.00   0.00    0.00    0.00   0.00
emma        0.00     0.00   0.00    0.00    0.00   1.52
isabella    0.00     0.00  11.82    0.00    0.00   0.00
liam        3.33     0.00   0.00    0.00    0.00   0.00
lucas       0.00    48.66   0.00    0.00    0.00   0.00
mason       0.00     0.00   0.00    0.00    0.00  14.84
mia         0.00     0.00   0.00    0.00  178.61   0.00
sophia      0.00     0.00   0.00  130.69    0.00   0.00

## Next steps

This notebook showed how [Opvious](https://www.opvious.io) can be used to define and apply optimization to concrete data, highlighting a few key features along the way (declarative modeling, remote solving, multi-objective support).

Check out the [SDK's documentation](https://opvious.readthedocs.io) to learn more or give one of the following extension ideas a try!

+ The formulation above assumes that all members involved in a transaction have equal share. How can we extend it to support arbitrary shares?
+ Which other multi-objective strategies would make sense and how would their solutions compare to the one above?
+ How can we extend the model to handle prior settlements (for example if Ava had already transferred money to Noah) or minimum transfer thresholds?